## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
from keras.datasets import cifar10
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

# 防止調用 GPU 報錯：Blas GEMM launch failed
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.per_process_gpu_memory_fraction = 0.8 #佔用 GPU 30% 的記憶體資源
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3)) 
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [3]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('-------- original ------------')
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

"""
# 資料增強 ==> train
batch_size=10   # 增強圖片每多少個批次產生
append_rows = 100  # 增加多少張的資料
images = x_train[:append_rows]
data_generator = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    width_shift_range=0.1,
    height_shift_range=0.1)
batch_times = int(append_rows / batch_size)+1
for i in range(1,batch_times):
    augmented_iamges = next(data_generator.flow(images, shuffle=False, batch_size=batch_size))
    x_train = np.append(x_train, augmented_iamges, axis=0)
y_train = np.append(y_train, y_train[:append_rows]) #結果也要加入相同張數的結果  
    
print('-------- after append ------------')
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
"""  

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

-------- original ------------
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [6]:
batch_size = 512 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 50個循環

model.compile(loss='binary_crossentropy',
              optimizer=Adam(0.01),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 50s 999us/step - loss: 0.4206 - acc: 0.9343 - val_loss: 0.4762 - val_acc: 0.9009
Epoch 2/30
50000/50000 [==============================] - 43s 850us/step - loss: 0.2502 - acc: 0.9421 - val_loss: 0.4410 - val_acc: 0.9023
Epoch 3/30
50000/50000 [==============================] - 43s 853us/step - loss: 0.2187 - acc: 0.9453 - val_loss: 0.4046 - val_acc: 0.8974
Epoch 4/30
50000/50000 [==============================] - 43s 853us/step - loss: 0.2045 - acc: 0.9475 - val_loss: 0.3399 - val_acc: 0.9099
Epoch 5/30
50000/50000 [==============================] - 43s 852us/step - loss: 0.1974 - acc: 0.9492 - val_loss: 0.4667 - val_acc: 0.8745
Epoch 6/30
50000/50000 [==============================] - 43s 853us/step - loss: 0.1890 - acc: 0.9505 - val_loss: 0.3423 - val_acc: 0.9067
Epoch 7/30
50000/50000 [==============================] - 43s 853us/step - loss: 0.1808 - acc: 0.9527 - val_loss: 0.